In [2]:
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import numpy as np
from scipy.ndimage import label
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
import os

In [4]:
img_folder = './img/'
label_folder = './label/'

for img_name in os.listdir(img_folder):
    if img_name.endswith('.nii'):

        # 0. 调用 totalsegmentator 进行分割
        input_file = os.path.join(img_folder, img_name)
        output_folder = os.path.join(label_folder, img_name.split('.')[0])
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        input_img = nib.load(input_file)
        output_img = totalsegmentator(input=input_img, output=output_folder, task="total_mr", roi_subset=['vertebrae'])
        print(f"分割结果已保存至: {output_folder}")
        
        # 1. 加载 NIfTI 图像
        input_path = output_folder + '/vertebrae.nii.gz'  # 输入 NIfTI 文件路径
        img = nib.load(input_path)
        data = img.get_fdata()  # 获取图像数据为 NumPy 数组
        binary_data = (data > 0).astype(np.int32) # 二值化图像数据

        # 2. 对值为1的区域进行连通区域标记
        labeled_array, num_features = label(binary_data)
        output_path = f'{output_folder}/vertebrae{num_features}.nii.gz'  # 输出 NIfTI 文件路径

        # 3. 获取所有连通区域的重心
        centers = center_of_mass(binary_data, labeled_array, range(1, num_features + 1))
        centers_array = np.array(centers)

        # 4. 使用 KMeans 聚类算法对重心进行聚类 聚类为 num_features 类
        kmeans = KMeans(n_clusters=num_features, random_state=0)
        kmeans.fit(centers_array)
        labels = kmeans.labels_

        # 5. 根据中心的z轴位置排序
        sorted_indices = np.argsort(-centers_array[:, 2])
        sorted_labels = np.zeros_like(labels)
        for i, index in enumerate(sorted_indices):
            sorted_labels[labels == index] = i

        # 6. 创建新的分割图像，将每个旧聚类的值替换为新的类标签
        clustered_data = np.zeros_like(binary_data)
        for oldlabel, newlabel in zip(labels, sorted_labels):
            clustered_data[labeled_array == (oldlabel + 1)] = newlabel + 1

        # 7. 保存新的分割图像
        new_img = nib.Nifti1Image(clustered_data, img.affine, img.header)
        nib.save(new_img, output_path)
        print(f"聚类后的图像已保存至: {output_path}")


If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.58s
Predicting...


100%|██████████| 1/1 [00:00<00:00, 164.42it/s]


  Predicted in 6.99s
Resampling...
  cropping from (196, 512, 256) to (196, 236, 235)
Resampling...
  Resampled in 0.23s
Predicting part 1 of 1 ...


100%|██████████| 2/2 [00:00<00:00, 166.42it/s]


  Predicted in 7.45s
Resampling...
Saving segmentations...


...

  Predicted in 6.67s
Resampling...
  cropping from (196, 512, 256) to (196, 236, 235)
Resampling...
  Resampled in 0.23s
Predicting part 1 of 1 ...


100%|██████████| 2/2 [00:00<00:00, 166.54it/s]


  Predicted in 7.46s
Resampling...
Saving segmentations...


  0%|          | 0/1 [00:00<?, ?it/s]

  Saved in 3.44s
分割结果已保存至: ./label/MRT1_0028_0001
聚类后的图像已保存至: ./label/MRT1_0028_0001/vertebrae7.nii.gz
